<h1>A Convolutional Neural Network using tensorflow</h1>
<p>This example also uses MNIST data to create CNN that will recognize hand-written digits.
Our expecation is that the results will be improved by using this approach versus the simple neural network.</p>
<p>The image data looks like this:</p>

<img src="../img/MNIST.png">


In [ ]:
# First import tensorflow and establish our session object
import tensorflow as tf
session = tf.InteractiveSession()

In [ ]:
# Now make sure we have the MNIST data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
# Here we establish the number of pixles per image and the range of digits (0 through 9) that we expect on each image.

width = height = 28
flat = width * height # = number of pixels
class_output = 10

In [ ]:
# Creating placeholders for input and output

X  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [ ]:
# This function will be responsible for generating weights having some noise to prevent the vanishing gradient problem.

def weight_variable(shape):
    weight = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weight)


In [ ]:
# Here is a function for generating a bias variable to avoid "dying" of neurons

def bias_variable(shape):
    bias = tf.constant(0.1, shape=shape)
    return tf.Variable(bias)

<h2> Here is the structure (layers) of our CNN:<h2>
<ol>
    <li>Input data</li>
    <li>Convolution and max pooling</li>
    <li>Convolution and max pooling</li>
    <li>Fully connected layer</li>
    <li>Processing - DropOut</li>
    <li>Readout layer - Fully Connected</li>
    <li>Output - Classified digits</li>
</ol>



In [ ]:
# Let's create a function for our convolution layers

def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')

In [ ]:
# And a function for 2x2 max pooling

def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
# Here we make our first convolution and max pooling layer

# Weights and Biases
W_conv1 = weight_variable([5, 5, 1, 32]) # Size of the filter/kernel: 5x5; Input channels: 1 (greyscale); 32 feature maps
b_conv1 = bias_variable([32]) # need 32 biases for 32 outputs

# Converting images of dataset to tensors
X_image = tf.reshape(X, [-1, 28, 28, 1])

# Convolving the weight tensor and adding bias
convolve1 = conv2d(X_image, W_conv1) + b_conv1

# Appling ReLu activation
h_conv1 = tf.nn.relu(convolve1)

# Applying max pooling
h_pool1 = max_pool_2x2(h_conv1)

layer1 = h_pool1

In [ ]:
# And here we make our second convolution and max pooling layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

convolve2 = conv2d(layer1, W_conv2) + b_conv2

h_conv2 = tf.nn.relu(convolve2)

h_pool2 = max_pool_2x2(h_conv2)

layer2 = h_pool2

In [ ]:
# Making the third layer - Fully connected layer to use softmax and create probabilities in the end.

# Weights and bias
# Last layer produce a feature map of 7x7, number of feature map = 64, output to softmax = 1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Flattening second layer
layer2_matrix = tf.reshape(layer2, [-1, 7 * 7 * 64])

# Applying weights and bias
matmul_fc1 = tf.matmul(layer2_matrix, W_fc1) + b_fc1

# Applying ReLu
h_fc1 = tf.nn.relu(matmul_fc1)

layer3 = h_fc1

In [ ]:
# Droput - regularization

keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

In [ ]:
# Final layer - Read out layer (softmax, fully connected)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

matmul_fc2 = tf.matmul(layer3_drop, W_fc2) + b_fc2

# Applying softmax activation
y_conv = tf.nn.softmax(matmul_fc2)
layer4 = y_conv

In [ ]:
#  Remember the cross entropy function from the Simple Neural Network?
#  Let's apply something like that to layer 4

cross_entropy = # TODO:  fill in code here

In [ ]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [ ]:
correct_prediction = tf.equal(tf.argmax(layer4, 1), tf.argmax(y_, 1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:

%timeit session.run(tf.global_variables_initializer())

In [ ]:
# Now the training:  Again, let's start to train the CNN using mini batches of 10 for 100 iterations.
# Once you see those results, increase both batch size and number of interations incrementally to 50/1000 and compare results.
ITERATIONS =  # TODO: fill in here
BATCHSIZE =   # TODO  fill in here

for _ in range(ITERATIONS):
    batch = mnist.train.next_batch(BATCHSIZE)
    if _ % 100 == 0:
        train_accuracy = accuracy.eval(session=session, feed_dict={X: batch[0], y_: batch[1], keep_prob: 1.0})
        print ("Step %d : %.5f" % (_ , float(train_accuracy)))
    train_step.run(session=session, feed_dict={X: batch[0], y_: batch[1], keep_prob: 0.5})

In [ ]:
print ("test accuracy %g"%accuracy.eval(session=session, feed_dict={X: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))